In [ ]:
# Until v4.26.0 is out, requires:
# pip install --upgrade git+https://github.com/huggingface/transformers
# Sometimes, I may have to use bleeding edge:
# pip install -U git+https://github.com/huggingface/diffusers

import random
import time

import diffusers
import numpy as np
import torch
import PIL

# To test performance delta.
use_cuda = torch.cuda.is_available()

def inference(prompt, n_images, guidance, steps, seed, img, strength, neg_prompt):
    """Use Depth-to-Image Diffusion Model as described at
    https://stability.ai/blog/stable-diffusion-v2-release
    """
    generator = None
    if use_cuda:
        generator = torch.Generator("cuda").manual_seed(seed) if seed else None
    elif seed:      
        generator = torch.Generator()
        generator.manual_seed(seed)
    pipe = diffusers.StableDiffusionDepth2ImgPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-depth",
        revision="fp16" if use_cuda else "fp32",
        torch_dtype=torch.float16 if use_cuda else torch.float32)
    pipe.scheduler = diffusers.DPMSolverMultistepScheduler.from_config(
        pipe.scheduler.config)
    if use_cuda:
        pipe.to("cuda")
        pipe.enable_attention_slicing()

    # See the code at:
    # https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_depth2img.py
    return pipe(
        prompt,
        num_images_per_prompt=n_images,
        negative_prompt=neg_prompt,
        image=img,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator).images

def run(prompt, image):
    n_images = 1 # number of images
    neg_prompt = "Disfigured. Too many fingers."
    guidance = 7.5 # max = 15
    steps = 50 # [2, 100]
    seed = 10 # random
    strength = 0.75
    # seed = random.randint(0, 2147483647)
    out = "%s_steps%d_strength%.2f_guidance%.1f_seed%d.png" % (prompt.replace(".", ""), steps, strength, guidance, seed)
    print(f"Seed: {seed}")
    start = time.time()
    gallery = inference(prompt, n_images, guidance, steps, seed, image, strength, neg_prompt)
    print("Took %.1fs" % (time.time()-start))
    gallery[0].save("out/"+out)
    return gallery[0]

def getimg():
    name = "bench.jpg"
    name = "PXL_20221117_233124192.PORTRAIT.jpg"
    name = "PXL_20221127_135034465.PORTRAIT.jpg"
    img = PIL.Image.open("out/" + name)
    size = img.size
    # Max is 1024x768 or 768x1024?
    while size[0] > 1024 or size[1] > 1024: # or size[0] * size[1] > 786432:
        size = (size[0]//2, size[1]//2)
    if size != img.size:
        print("Resized from", img.size, "to", size)
        img = img.resize(size, PIL.Image.Resampling.LANCZOS)
    return img

#  A whimsical illustration of a robot picking flowers. The robot is depicted as a cute and playful machine, with big, round eyes and a cheerful smile. It is shown holding a bouquet of colorful flowers, and it is surrounded by a lush and vibrant garden. The illustration is done in a bright and cheerful style, and it captures the joy and wonder of the robot's experience.
img = run("Christmas", getimg())
img